In [7]:
import pandas as pd
import numpy as np

In [ ]:
import requests
import pandas as pd
from datetime import datetime

# Ambil tanggal sistem
current_date = datetime.now().strftime('%Y-%m-%d')

# Buat URL
url = f"https://svc-silinda.jabarprov.go.id//api/api/graphic_data/1/69/day/price/2021-03-28/{current_date}/0/market/-/eceran/null"

# Request ke API
response = requests.get(url)

# Cek apakah sukses
if response.status_code == 200:
    data = response.json()
    
    # Pastikan data yang diambil adalah list atau dict yang bisa diubah jadi DataFrame
    if isinstance(data, list):
        df = pd.DataFrame(data)
    elif isinstance(data, dict):
        # Jika berbentuk dict, ambil isi yang relevan
        if 'data' in data:
            df = pd.DataFrame(data['data'])
        else:
            df = pd.DataFrame([data])
    
    # Simpan ke CSV
    filename = f"data_silinda.csv"
    df.to_csv(filename, index=False)
    print(f"Data berhasil disimpan ke {filename}")
else:
    print(f"Request gagal dengan status code {response.status_code}")





In [ ]:
df =  pd.read_csv('silinda_data.csv')

,total,result,average,location_id,commodity_id,legend,commodity_name
0,1859324,"[{'value': '10500', 'time': 'June 11, 2021', '...",12823,69,1,Harga Pasar Cikurubuk,Beras Medium


In [11]:
import ast
from datetime import datetime

def extract_result_data(df):
    """
    Fungsi untuk mengekstrak dan membersihkan data dari kolom 'result'
    """
    try:
        # 1. Konversi string ke list of dicts
        if isinstance(df['result'], str):
            try:
                # Ganti tanda kutip tunggal dengan ganda untuk valid JSON
                json_str = df['result'].replace("'", '"')
                results_list = ast.literal_eval(json_str)
            except (SyntaxError, ValueError) as e:
                print(f"Error parsing JSON: {e}")
                return pd.DataFrame()  # Return empty DataFrame jika error
        
        # 2. Konversi ke DataFrame
        df_results = pd.DataFrame(results_list)
        
        if not df_results.empty:
            # 3. Pembersihan data
            # Konversi tipe data
            df_results['value'] = pd.to_numeric(df_results['value'], errors='coerce')
            
            # Handle multiple date formats
            df_results['date'] = pd.to_datetime(
                df_results['date'], 
                errors='coerce',
                format='mixed'  # Untuk handle berbagai format tanggal
            )
            
            # Drop baris dengan data hilang
            df_results.dropna(subset=['value', 'date'], inplace=True)
            
            # 4. Sort by date
            df_results.sort_values('date', inplace=True)
            
            return df_results
            
    except Exception as e:
        print(f"Unexpected error: {e}")
    
    return pd.DataFrame()

# Contoh penggunaan:
if not df.empty and 'result' in df.columns:
    # Ekstrak data untuk setiap baris (jika df memiliki multiple rows)
    all_results = pd.concat(
        [extract_result_data(row) for _, row in df.iterrows()],
        ignore_index=True
    )
    
    print("DataFrame Hasil Ekstraksi:")
    print(all_results.head())
    
    # Simpan ke CSV
    try:
        all_results.to_csv('hasil_ekstraksi.csv', index=False, encoding='utf-8-sig')
        print("Data berhasil disimpan ke hasil_ekstraksi.csv")
    except Exception as e:
        print(f"Gagal menyimpan file: {e}")
else:
    print("DataFrame input tidak valid atau kolom 'result' tidak ditemukan")


DataFrame Hasil Ekstraksi:
   value               time       date    span
0  10500      June 11, 2021 2021-06-11  202123
1  10000  February 11, 2022 2022-02-11  202206
2  10000  February 18, 2022 2022-02-18  202207
3  10000  February 25, 2022 2022-02-25  202208
4  10000      March 4, 2022 2022-03-04  202209
Data berhasil disimpan ke hasil_ekstraksi.csv
